# Some example SDFITSIO operations with masking


In [ ]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt


import sdfitsio as sdf
from importlib import reload
reload(sdf);

Create a masked array 'a' with a typical LMT structure

In [ ]:
ntime = 1000
nbeam = 16
npol = 2
nband = 6
nchan = 256

adim=(ntime,nbeam,npol,nband,nchan)
%time a = sdf.gen_data(adim)

print('Size of data',sdf.dimsize(adim))

Write it as SDFITS

In [ ]:
%time sdf.my_write_sdfits('junk.fits',a)

Set up a masking operation. Low level method. Any dimensions skipped, imply all elements are selected, i.e. [:]

In [ ]:
mask = {}
mask[0] = '0:110'     # time 
#mask[1] = '3'        # beam
#mask[2] = '0'        # pol
#mask[3] = '4'        # band
mask[4] = '81:83'    # chan


In [ ]:
%time sdf.data_mask(a, mask)


Report what fraction was masked

In [ ]:
%time sdf.data_masked(a)

In [ ]:
%time sdf.my_write_sdfits('junk_masked.fits',a)

Read back the original unmasked

In [ ]:
%time b=sdf.my_read('junk.fits')

In [ ]:
b.shape

In [ ]:
%time sdf.data_masked(b)

In [ ]:
%time c = sdf.my_read('junk_masked.fits')

In [ ]:
%time sdf.data_masked(c)

Indeed, the masking have now been applied (as NaN) in the SDFITS file. masks cannot be undone here, since we did not save the masks. Thus users are encouraged to save them seperately.

In [ ]:
x=np.arange(nchan)
y=b[0,0,0,0,:]
z=b.mean(axis=0).mean(axis=0).mean(axis=0).mean(axis=0)
plt.plot(x,y,label='1 scan')
plt.plot(x,z,label='stacked')
plt.legend()
ys=y[:100].std()
zs=z[:100].std()
bdims=b.shape
print(ys,zs,ys/zs,np.sqrt(sdf.dimsize(bdims[:-1])))



In [ ]:
%time s = sdf.Spectra(adim)

In [ ]:
%time s.aver()



In [ ]:
x=np.arange(nchan)
y=s.data
plt.plot(x,y);